<!-- -*- mode: markdown; coding: utf-8; fill-column: 60; ispell-dictionary: "english" -*- -->

<meta charset="utf-8"/>
<meta name="viewport" content="width=device-width,initial-scale=1"/>
<link rel="stylesheet" href="style.css">


# EDAF75 - lab 2: Testing the database

As usual we have to tell Jupyter to allow SQL:

In [1]:
%load_ext sql

And then we import our movie database

In [2]:
%sql sqlite:///movies.sqlite

We want to make sure that SQLite3 really checks our foreign
key constraints -- to do that, we run:

In [3]:
%%sql
PRAGMA foreign_keys=ON;

 * sqlite:///movies.sqlite
Done.


[]

Now write SQL code for the following tasks:


+ Show the names of all movies.

In [4]:
%%sql
SELECT title
FROM movies 


 * sqlite:///movies.sqlite
Done.


title
Inception
The Godfather
Interstellar
The Dark Knight
Pulp Fiction
Forrest Gump
The Matrix


+ Show the performance dates for one of the movies.

In [5]:
%%sql
SELECT DISTINCT date 
FROM movies JOIN performances
WHERE title = "Inception"

 * sqlite:///movies.sqlite
Done.


date
2023-04-10
2023-04-11
2023-04-12
2023-04-13
2023-04-14
2023-04-15
2023-04-16
2023-04-17
2023-04-18
2023-04-19


+ Show all data concerning performances at a given theatere
  on a given date.

In [25]:
%%sql
SELECT title, date, start_time, run_time
FROM performances LEFT OUTER JOIN movies USING (imdb)
WHERE date = '2023-04-17' AND theater_name = 'Sergel'

 * sqlite:///movies.sqlite
Done.


title,date,start_time,run_time
The Godfather,2023-04-17,17:00,175
Pulp Fiction,2023-04-17,23:00,154


+ List all customers

In [11]:
%%sql
SELECT username, full_name
FROM customers

 * sqlite:///movies.sqlite
Done.


username,full_name
Fabian,Fabian Rosen
josfri,Josefine Frid
Fremja,Fremja Ekre
leoDicap,Leonardo DiCaprio
beyKnow,Beyonce Knowles
tomCruise,Tom Cruise
tSwift,Taylor Swift
rdjIron,Robert Downey Jr.
scarJo,Scarlett Johansson
chrEvans,Chris Evans


+ List all tickets

In [29]:
%%sql
SELECT ticket_id, full_name, title, start_time, theater_name
FROM tickets JOIN performances USING(performance_id) JOIN movies USING(imdb) JOIN customers USING (username)

 * sqlite:///movies.sqlite
Done.


ticket_id,full_name,title,start_time,theater_name
18522cdaa1d80d18da6938213c724e99,Leonardo DiCaprio,Interstellar,23:00,Sergel
cbef188c11313eaebfa061845560d1cb,Beyonce Knowles,Interstellar,23:00,Sergel
50f773eef0252dabee8f890227648cd5,Tom Cruise,Interstellar,23:00,Sergel
ce1259540ebd7edf2107ebafa3a1834d,Taylor Swift,Interstellar,19:00,Rigoletto
d9d55e7c4d32908485348f15aca0afaf,Robert Downey Jr.,Interstellar,19:00,Rigoletto
7aefa204542597c45b1432168b5e42fc,Scarlett Johansson,Interstellar,19:00,Rigoletto
79a7b97c138fdee51ebc7bcb29f67b24,Chris Evans,Interstellar,19:00,Rigoletto
ca62c8c4752174e109c66ab8419273a5,Emma Stone,The Dark Knight,21:00,Grand Stockholm
233e1b4ccfdda602fa3443eacdcd78b9,Johnny Depp,The Dark Knight,21:00,Grand Stockholm
e15a88315ff23e0f06f5900420280873,Jennifer Aniston,The Dark Knight,21:00,Grand Stockholm


+ Create a new ticket to some performance (i.e., insert a
  new row in your table of tickets).

In [28]:
%%sql
INSERT 
INTO  tickets (username, performance_id)
VALUES   ('josfri', '7e9a97af3eb4a66c3c1b9bd4a0e88f98')
RETURNING  ticket_id

 * sqlite:///movies.sqlite
0 rows affected.


ticket_id
4993bd26b893197c74dab02a52b540c7


In newer versions of SQLite (since version 3.35, released
  in March 2021), and in
  [PostgreSQL](https://www.postgresql.org/docs/current/sql-insert.html),
  we can get any value generated during an insert using the
  `INSERT...-RETURNING` statement:

~~~{.sql}
INSERT
INTO       students
VALUES     ('Amy', 3.9, 1200)
RETURNING  s_id
~~~


which would return the generated `s_id` for the new
  student.

  If your SQLite version is older than 3.35, and you can't
  upgrade, you can instead use the following idea: each row
  in a SQLite3 table has a `rowid` attribute, it is a unique
  integer which essentially tells in which order the rows
  were inserted, and it's not displayed in queries unless we
  ask for it. SQLite3 also have a function,
  `last_insert_rowid()`, which returns the `rowid` of the
  last inserted row of a table, so we can see the `s_id` of
  the most recently inserted student with the following
  query:

~~~{.sql}
SELECT s_id
FROM   students
WHERE  rowid = last_insert_rowid();
~~~


Now, check what ticket number we got for the ticket we
  created above (it should be the same as the ticket id,
  which should be a `randomblob`):

In [30]:
%%sql
SELECT ticket_id
FROM tickets
WHERE username = 'josfri'

 * sqlite:///movies.sqlite
Done.


ticket_id
4993bd26b893197c74dab02a52b540c7


+ Try to insert two movie theaters with the same name (this
  should fail).

In [37]:
%%sql
INSERT
INTO theaters (theater_name, capacity) VALUES
('Grand Danderyd', 50),
('Grand  Danderyd', 100);

 * sqlite:///movies.sqlite
(sqlite3.IntegrityError) UNIQUE constraint failed: theaters.theater_name
[SQL: INSERT
INTO theaters (theater_name, capacity) VALUES
('Grand Danderyd', 50),
('Grand  Danderyd', 100);]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


+ Try to insert a performance where the theater doesnâ€™t
  exist in the database (this should fail).

In [41]:
%%sql
INSERT
INTO performances (theater_name, start_time, date, imdb) VALUES
('Grand Lidingö', '19:00', '2023-01-15', 'tt0068646')

 * sqlite:///movies.sqlite
1 rows affected.


[]

+ Create a ticket where either the user or the performance
  doesnâ€™t exist (this should fail).

In [45]:
%%sql
INSERT 
INTO  tickets (username, performance_id)
VALUES   ('tomhanks', '7e9a97af3eb4a66c3c1b9bd4a0e88f98')
RETURNING  ticket_id

 * sqlite:///movies.sqlite
(sqlite3.IntegrityError) FOREIGN KEY constraint failed
[SQL: INSERT 
INTO  tickets (username, performance_id)
VALUES   ('tomhanks', '7e9a97af3eb4a66c3c1b9bd4a0e88f98')
RETURNING  ticket_id]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


In [44]:
%%sql
INSERT 
INTO  tickets (username, performance_id)
VALUES   ('josfri', '7e9a97af3eb4a66c3c1b9bd4a0e11f98')
RETURNING  ticket_id

 * sqlite:///movies.sqlite
(sqlite3.IntegrityError) FOREIGN KEY constraint failed
[SQL: INSERT 
INTO  tickets (username, performance_id)
VALUES   ('josfri', '7e9a97af3eb4a66c3c1b9bd4a0e11f98')
RETURNING  ticket_id]
(Background on this error at: https://sqlalche.me/e/20/gkpj)
